<a href="https://colab.research.google.com/github/iagomauricioo/data_science/blob/main/mapa_marechal_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import folium
import pandas as pd
from geopy.distance import geodesic
from IPython.display import HTML

In [2]:
barra_sao_miguel_coords = [-9.8379, -35.8979]
marechal_deodoro_coords = [-9.7092, -35.8964]

In [3]:
center_lat = (barra_sao_miguel_coords[0] + marechal_deodoro_coords[0]) / 2
center_lon = (barra_sao_miguel_coords[1] + marechal_deodoro_coords[1]) / 2
center_coords = [center_lat, center_lon]

In [4]:
m = folium.Map(location=center_coords, zoom_start=12)

In [5]:
folium.Marker(location=barra_sao_miguel_coords, popup='Barra de São Miguel', icon=folium.Icon(color='blue'), zIndexOffset=1000).add_to(m)
folium.Marker(location=marechal_deodoro_coords, popup='Marechal Deodoro', icon=folium.Icon(color='green'), zIndexOffset=1000).add_to(m)

In [6]:
radius = 10000  # 10 km em metros
folium.Circle(location=center_coords, radius=radius, color='red', fill=True, fill_color='red', fill_opacity=0.2).add_to(m)

In [7]:
m

# Área onde será realizado o mapa de calor em breve.

In [7]:
arquivo_excel = '/content/drive/MyDrive/coordenadas.xlsx'
#arquivo_excel = '/content/coordenadas (1).xlsx'
df = pd.read_excel(arquivo_excel)
df.dropna(inplace=True)
df

,tipo,cidade,coordenadas
0,Mercado,Barra de São Miguel,"-9.840822033037991, -35.905050152891015"
1,Mercado,Barra de São Miguel,"-9.841170945190338, -35.904413953028474"
2,Mercado,Barra de São Miguel,"-9.843521890587615, -35.903697672912195"
3,Mercado,Barra de São Miguel,"-9.84291147460505, -35.9020740760202"
4,Mercado,Barra de São Miguel,"-9.842722965624427, -35.90114136570571"
...,...,...,...
77,Hotel/Pousada,Marechal Deodoro,"-9.767803798785666, -35.844507294832745"
78,Hotel/Pousada,Marechal Deodoro,"-9.767465452869695, -35.846749621552846"
79,Hotel/Pousada,Marechal Deodoro,"-9.767641466780077, -35.84837169226835"
80,Hotel/Pousada,Marechal Deodoro,"-9.767821212988707, -35.84831804808942"


In [8]:
data = df[['tipo', 'coordenadas']]
data[['latitude', 'longitude']] = data['coordenadas'].str.split(',', expand=True)
data['latitude'] = data['latitude'].astype(float)
data['longitude'] = data['longitude'].astype(float)
data = data.drop('coordenadas', axis=1)
data.dropna(inplace=True)
data

,tipo,latitude,longitude
0,Mercado,-9.840822,-35.905050
1,Mercado,-9.841171,-35.904414
2,Mercado,-9.843522,-35.903698
3,Mercado,-9.842911,-35.902074
4,Mercado,-9.842723,-35.901141
...,...,...,...
77,Hotel/Pousada,-9.767804,-35.844507
78,Hotel/Pousada,-9.767465,-35.846750
79,Hotel/Pousada,-9.767641,-35.848372
80,Hotel/Pousada,-9.767821,-35.848318


In [9]:
for _, row in data[data['tipo'] == 'Mercado'].iterrows():
    m.add_child(
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup="Tipo de estabelecimento: Mercado",
            icon=folium.Icon(color='purple')
        )
    )

# Comparação
Para efeitos de comparação, na primeira imagem abaixo podemos ver uma imagem diretamente do Google Maps. Já na segunda imagem, vemos um mapa feito em python com os locais traçados iguais ao google maps. (Apenas para vermos o quão parecido está para ter uma noção de margem de erros).

In [10]:
iframe_code = """
<iframe src="https://www.google.com/maps/embed?pb=!1m16!1m12!1m3!1d31449.56301026032!2d-35.90486405726632!3d-9.833951979960371!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!2m1!1ssupermercado!5e0!3m2!1spt-BR!2sbr!4v1718769099993!5m2!1spt-BR!2sbr" width="600" height="450" style="border:0;" allowfullscreen="" loading="lazy" referrerpolicy="no-referrer-when-downgrade"></iframe>
"""

HTML(iframe_code)


## Na imagem acima, vemos a quantidade de supermercados na Barra de São Miguel.

In [10]:
m

In [77]:
hotels = df[df['tipo'] == 'Hotel/Pousada']
hotels

,tipo,cidade,coordenadas
34,Hotel/Pousada,Barra de São Miguel,"-9.845913829636691, -35.904858178106075"
35,Hotel/Pousada,Barra de São Miguel,"-9.844032219592265, -35.90018040561898"
36,Hotel/Pousada,Barra de São Miguel,"-9.842319733663976, -35.89861399549438"
37,Hotel/Pousada,Barra de São Miguel,"-9.839507854871455, -35.897240704473305"
38,Hotel/Pousada,Barra de São Miguel,"-9.837541639674297, -35.89756256955618"
39,Hotel/Pousada,Barra de São Miguel,"-9.837605066140545, -35.89766985794075"
40,Hotel/Pousada,Barra de São Miguel,"-9.838366182950596, -35.89166170977518"
41,Hotel/Pousada,Barra de São Miguel,"-9.838027908991787, -35.8898592652567"
42,Hotel/Pousada,Barra de São Miguel,"-9.836442245383408, -35.88756329441352"
43,Hotel/Pousada,Barra de São Miguel,"-9.832044631845548, -35.88689810653476"


In [11]:
for _, row in data[data['tipo'] == 'Hotel/Pousada'].iterrows():
    m.add_child(
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup="Tipo de estabelecimento: Hotel/Pousada",
            icon=folium.Icon(color='orange')
        )
    )

In [15]:
legend_html = '''
     <div style="
     position: fixed;
     bottom: 50px; left: 50px; width: 200px; height: 200px;
     background-color: white; z-index:9999; font-size:14px;
     border:2px solid grey; padding: 10px;
     ">
     <h4>Legenda</h4>
     <i class="fa fa-map-marker" style="color:blue"></i>Barra de São Miguel<br>
     <i class="fa fa-map-marker" style="color:green"></i>Marechal<br>
     <i class="fa fa-map-marker" style="color:orange"></i>Hotel/Pousada<br>
     <i class="fa fa-map-marker" style="color:purple"></i>Supermercado<br>
     <i class="fa fa-map-marker" style="color:red"></i>Restaurante/Bar<br>
     </div>
     '''

In [16]:
m.get_root().html.add_child(folium.Element(legend_html))

In [17]:
m